# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [22]:
# import libraries
import re
import pickle
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt', 'wordnet','stopwords'])

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, make_scorer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql(sql="SELECT * FROM disaster", con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)
category_names = Y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    
    # Convert text to lower case:
    text = text.lower()
    
    # Remove punctuation chars:
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenization:
    tokenized = word_tokenize(text=text)
    
    # Remove stopwords:
    tokenized = [w for w in tokenized if(w not in stopwords.words("english"))]
    
    # Lemmatization:
    tokenized = [WordNetLemmatizer().lemmatize(w) for w in tokenized]
    
    return tokenized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
pipeline = Pipeline(steps=[
    ("vect", CountVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
# Split into train/test:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

In [15]:
X_test.iloc[8]

'On January 21, it signed a four-year, $200 million agreement with the GOP to support the earthquake reconstruction program.'

In [16]:
Y_test.iloc[8]

related                   1
request                   0
offer                     0
aid_related               1
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     1
missing_people            0
refugees                  0
death                     0
other_aid                 1
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           1
floods                    0
storm                     0
fire                      0
earthquake                1
cold                      0
other_weather             0
direct_report       

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
y_pred = pipeline.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], y_pred[:, i]))

related
             precision    recall  f1-score   support

          0       0.60      0.45      0.52      1873
          1       0.84      0.90      0.87      5934
          2       0.43      0.31      0.36        58

avg / total       0.78      0.79      0.78      7865

request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.78      0.45      0.57      1332

avg / total       0.88      0.89      0.87      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      4646
          1       0.73      0.62      0.67      3219

avg / total       0.75      0.75      0.75      7865

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [29]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fa25a33c378>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=42, v

In [48]:
parameters = {
    "clf__estimator__max_depth": [1, 3],
    "clf__estimator__n_estimators": [50, 100]
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=-1,
                  cv=5, verbose=2)

In [ ]:
# Training pipeline:
cv.fit(X_train, Y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total= 1.5min
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total= 1.5min
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....
[CV]  clf__estimator__max_depth=1, clf__estimator__n_estimators=50, total= 1.5min
[CV] clf__estimator__max_depth=1, clf__estimator__n_estimators=50 ....


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_cv = pipeline.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], y_pred_cv[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# AdaBoost Classifier:
pipeline2 = Pipeline(steps=[
    ("vect", CounVectorizer(tokenizer=tokenize)),
    ("tfidf", TfidfTransformer()),
    ("clf", MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
# Training pipeline2:
pipeline2.fit(X_train, Y_train)

In [ ]:
y_pred_ada = pipeline2.predict(X_test)
for i in range(len(category_names)):
    category = category_names[i]
    print(category)
    print(classification_report(Y_test[category], y_pred_ada[:, i]))

### 9. Export your model as a pickle file

In [ ]:
model_pkl = "ada.pkl"
with open(model_pkl, "wb") as file:
    pickle.dump(pipeline2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.